# PreProcess

In [ ]:
import io
import re
import string
import tqdm

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

In [ ]:
#Read DataSet and Stop words

with open ("hafez.txt", "r") as dataset:
    data = dataset.read().splitlines()
    text_ds = tf.data.TextLineDataset(dataset.read()).filter(lambda x: tf.cast(tf.strings.length(x), bool))

with open ("fa_stop_words.txt", "r") as fa_stop_words:
    stop_words = fa_stop_words.read().splitlines()    

In [ ]:
print("data:", "\n1: ", data[0], "\n2: ", data[1], "\n3: ", data[2])
print("\nstop_words:", "\n1: ", stop_words[18], "\n2: ", stop_words[19], "\n3: ", stop_words[20])

data: 
1:  ﻿ 
2:  الا يا ايها الساقي ادر كاسا و ناولها 
3:  كه عشق آسان نمود اول ولي افتاد مشكل‌ها

stop_words: 
1:  ؟ 
2:  آباد 
3:  آخ
<FilterDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>




---



In [ ]:
!pip install hazm

from hazm import *
import numpy as np
import random
import pandas as pd
import tensorflow as tf

In [ ]:
normalizer = Normalizer()
stemmer = Stemmer()
normalized_data = []
tokenized_data = []
clear_data = []
stemmed_data = []
clear_sentences = []
words = []
clear_words = []

#### Normalize
for line in data:
  if line and line != '\ufeff':
    normalized_data.append(normalizer.normalize(line))


#### Tokenize
for line in normalized_data:
  tokenized_data.append(word_tokenize(line))


#### Remove stop words
for line in tokenized_data:
  clear_line = []
  for word in line:
    if word not in stop_words:
      clear_line.append(word)
  clear_data.append(clear_line)



#### Stemize
for line in clear_data:
  stemmed_line = []
  for word in line:
    stemmed_line.append(stemmer.stem(word))
  stemmed_data.append(stemmed_line)


#### Create Bag of words
for i in range(len(stemmed_data)):
  words.extend(stemmed_data[i])


#### Convert words to sentences
for line in stemmed_data:
  clear_sentences.append(" ".join(line))
clear_sentences = [item for item in clear_sentences if item]


# Remove duplicate words
for i in words: 
  if i not in clear_words: 
    clear_words.append(i) 


print("Normalized: ", normalized_data[0])
print("Tokenized: ", tokenized_data[0])
print("Cleared: ", clear_data[0])
print("Words: ", words[0], "size: ", len(words))
print("Non Duplicate Words: ", clear_words[0], "size: ", len(clear_words))
print("Sentence: ", clear_sentences[0])

Normalized:  الا یا ایها الساقی ادر کاسا و ناولها
Tokenized:  ['الا', 'یا', 'ایها', 'الساقی', 'ادر', 'کاسا', 'و', 'ناولها']
Cleared:  ['الساقی', 'ادر', 'کاسا', 'ناولها']
Words:  الساق size:  34185
Non Duplicate Words:  الساق size:  5760
Sentence:  الساق ادر کاسا ناول


# Process

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

sequences = tokenizer.texts_to_sequences(clear_sentences)
max_len = 100

inverse_vocab = pad_sequences(sequences, padding = 'pre', maxlen= max_len)
print(inverse_vocab)

[[   0    0    0 ... 2990 2135 2991]
 [   0    0    0 ...  587  150  384]
 [   0    0    0 ...   40  215  946]
 ...
 [   0    0    0 ...    0   28  101]
 [   0    0    0 ... 2782  446   44]
 [   0    0    0 ...    0   94    2]]


In [ ]:
# Generates skip-gram pairs with negative sampling for a list of sequences
# (int-encoded sentences) based on window size, number of negative samples
# and vocabulary size.
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
  # Elements of each training example are appended to these lists.
  targets, contexts, labels = [], [], []

  # Build the sampling table for `vocab_size` tokens.
  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  # Iterate over all sequences (sentences) in the dataset.
  for sequence in tqdm.tqdm(sequences):

    # Generate positive skip-gram pairs for a sequence (sentence).
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence,
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=0)

    # Iterate over each positive skip-gram pair to produce training examples
    # with a positive context word and negative samples.
    for target_word, context_word in positive_skip_grams:
      context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)
      negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1,
          num_sampled=num_ns,
          unique=True,
          range_max=vocab_size,
          seed=SEED,
          name="negative_sampling")

      # Build context and label vectors (for one target word)
      negative_sampling_candidates = tf.expand_dims(
          negative_sampling_candidates, 1)

      context = tf.concat([context_class, negative_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_ns, dtype="int64")

      # Append each element from the training example to global lists.
      targets.append(target_word)
      contexts.append(context)
      labels.append(label)

  return targets, contexts, labels


In [ ]:
vocab_size = len(clear_words)


targets, contexts, labels = generate_training_data(
    sequences=sequences,
    window_size=2,
    num_ns=4,
    vocab_size=vocab_size,
    seed=SEED)

targets = np.array(targets)
contexts = np.array(contexts)[:,:,0]
labels = np.array(labels)

print('\n')
print(f"targets.shape: {targets.shape}")
print(f"contexts.shape: {contexts.shape}")
print(f"labels.shape: {labels.shape}")


100%|██████████| 8384/8384 [00:02<00:00, 2990.76it/s]




targets.shape: (15833,)
contexts.shape: (15833, 5)
labels.shape: (15833, 5)


In [ ]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

<BatchDataset element_spec=((TensorSpec(shape=(1024,), dtype=tf.int64, name=None), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None)), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None))>


In [ ]:
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset)

<PrefetchDataset element_spec=((TensorSpec(shape=(1024,), dtype=tf.int64, name=None), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None)), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None))>


In [ ]:
class Word2Vec(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = layers.Embedding(vocab_size,
                                      embedding_dim,
                                      input_length=1,
                                      name="w2v_embedding")
    self.context_embedding = layers.Embedding(vocab_size,
                                       embedding_dim,
                                       input_length=num_ns+1)

  def call(self, pair):
    target, context = pair
    # target: (batch, dummy?)  # The dummy axis doesn't exist in TF2.7+
    # context: (batch, context)
    if len(target.shape) == 2:
      target = tf.squeeze(target, axis=1)
    # target: (batch,)
    word_emb = self.target_embedding(target)
    # word_emb: (batch, embed)
    context_emb = self.context_embedding(context)
    # context_emb: (batch, context, embed)
    dots = tf.einsum('be,bce->bc', word_emb, context_emb)
    # dots: (batch, context)
    return dots

In [ ]:
def custom_loss(x_logit, y_true):
      return tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=y_true)

In [ ]:
embedding_dim = 128
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [ ]:
word2vec.fit(dataset, epochs=20, callbacks=[tensorboard_callback])

Epoch 1/20
15/15 [==============================] - 2s 32ms/step - loss: 1.6093 - accuracy: 0.2049
Epoch 2/20
15/15 [==============================] - 0s 31ms/step - loss: 1.5974 - accuracy: 0.6400
Epoch 3/20
15/15 [==============================] - 0s 30ms/step - loss: 1.5859 - accuracy: 0.8637
Epoch 4/20
15/15 [==============================] - 0s 29ms/step - loss: 1.5719 - accuracy: 0.9370
Epoch 5/20
15/15 [==============================] - 0s 21ms/step - loss: 1.5540 - accuracy: 0.9649
Epoch 6/20
15/15 [==============================] - 0s 20ms/step - loss: 1.5306 - accuracy: 0.9742
Epoch 7/20
15/15 [==============================] - 0s 20ms/step - loss: 1.5004 - accuracy: 0.9775
Epoch 8/20
15/15 [==============================] - 0s 21ms/step - loss: 1.4622 - accuracy: 0.9779
Epoch 9/20
15/15 [==============================] - 0s 21ms/step - loss: 1.4154 - accuracy: 0.9773
Epoch 10/20
15/15 [==============================] - 0s 20ms/step - loss: 1.3599 - accuracy: 0.9774
Epoch 11/